# 使用 RFM 模型对用户进行分层

> 完整代码与数据、结果：https://github.com/DarkestSky/SellingDataAnalysis

## 1. 读取 csv 文件并进行初步处理

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

# 针对日期格式进行处理
custom_date_parser = lambda x: datetime.strptime(x, "%d-%m-%Y %H:%M")

# 打开文件并通过 pandas 读取
path = r'./data/OnlineRetail.csv'
f = open(path, 'r', encoding='utf-8')
df = pd.read_csv(f, parse_dates=['InvoiceDate'], date_parser=custom_date_parser)

# 计算各条记录中的消费总额
df['Total'] = df['Quantity'] * df['UnitPrice']

# 记录月份
df['Month'] = df['InvoiceDate'].astype('datetime64[M]')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   CustomerID   406829 non-null  float64       
 1   Quantity     541909 non-null  int64         
 2   InvoiceDate  541909 non-null  datetime64[ns]
 3   UnitPrice    541909 non-null  float64       
 4   Total        541909 non-null  float64       
 5   Month        541909 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1)
memory usage: 24.8 MB
None


## 2. 参考 RFM 模型进行分层
首先是分层的标准，如下所示

In [12]:
def getRfmLable(x):
    # 传入参数为与平均值比较后的结果
    # 结果为正即记为 1
    level = x.map(lambda _x: '1' if _x >= 0 else '0')
    label = level.R + level.F + level.M
    d = {
        '111': u'重要价值客户',
        '011': u'重要保持客户',
        '101': u'重要挽留客户',
        '001': u'重要发展客户',
        '110': u'一般价值客户',
        '010': u'一般保持客户',
        '100': u'一般挽留客户',
        '000': u'一般发展客户'
    }
    result = d[label]
    return result

基于此，进行分层

In [13]:
# 透视图
rfm = df.pivot_table(index='CustomerID', aggfunc={'InvoiceDate': 'max', 'Quantity': 'sum', 'Total': 'sum'})

# 计算最后一次消费距离最后一天的间隔，即为 R
max_date = df['InvoiceDate'].max()
rfm['R'] = -(df.groupby(by='CustomerID')['InvoiceDate'].max() - max_date) / np.timedelta64(1, 'D')

# 调整表列
rfm.drop(columns=['InvoiceDate'], inplace=True)
rfm.rename(columns={'Quantity': 'F', 'Total': 'M'}, inplace=True)

# 与平均值比较，生成分层标签
rfm['Lable'] = rfm.apply(lambda x: x - x.mean()).apply(getRfmLable, axis=1)

print(rfm)

               F        M           R   Lable
CustomerID                                   
12346.0        0     0.00  325.106250  一般挽留客户
12347.0     2458  4310.00    1.873611  重要保持客户
12348.0     2341  1797.24   74.984028  一般保持客户
12349.0      631  1757.55   18.124306  一般发展客户
12350.0      197   334.40  309.867361  一般挽留客户
...          ...      ...         ...     ...
18280.0       45   180.60  277.123611  一般挽留客户
18281.0       54    80.82  180.081250  一般挽留客户
18282.0       98   176.60    7.046528  一般发展客户
18283.0     1397  2094.88    3.033333  重要保持客户
18287.0     1586  1837.28   42.139583  一般保持客户

[4372 rows x 4 columns]


得到分层信息，完整结果保存在 csv 文件中。